In [1]:
import openturns as ot
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from shapley.kriging import KrigingIndices, janon_estimator, compute_indice, compute_indices, create_df_from_gp_indices, create_df_from_indices
from shapley.tests.test_functions import ishigami, ot_ishigami, ishigami_true_indices
from shapley.plots import set_style_paper, violin_plot_indices

import time

%load_ext autoreload
%autoreload 2

%matplotlib inline

set_style_paper()

In [2]:
dim = 3
model = ishigami
ot_model = ot.PythonFunction(dim, 1, ot_ishigami)
margins = [ot.Uniform(-np.pi, np.pi)]*dim
copula = ot.IndependentCopula(dim)
input_distribution = ot.ComposedDistribution(margins, copula)
ot.RandomGenerator.SetSeed(0)

## Sobol indices with MC

In [ ]:
max_budgets = [50, 100, 200, 500, 1000]
n_samples_gp = [50, 100, 200, 500, 1000, 2000]
n_boot = 1000
n_realization_gp = 500
ylim = [-.2, 1.]
for n_sample_gp in max_budgets:
    for max_budget in max_budgets:
        t_start = time.time()
        print('n_sample_gp: %d, max_budget: %d' % (n_sample_gp, max_budget))
        n_sample = int(max_budget/(2*dim))
        input_sample_1 = np.asarray(input_distribution.getSample(n_sample))
        input_sample_2 = np.asarray(input_distribution.getSample(n_sample))
        first_indices_mc = compute_indices(model, input_sample_1, input_sample_2, n_boot=n_boot)

        kriging = KrigingIndices(ot_model, input_distribution)
        kriging.build_model(n_sample_kriging=max_budget)

        first_indices_gp = kriging.compute_indices(n_sample=n_sample_gp, n_realization=n_realization_gp, n_boot=n_boot)

        var_MC = first_indices_gp.std(axis=2).mean(axis=1)
        var_kriging = first_indices_gp.std(axis=1).mean(axis=1)
        print('MC var: {0}\nKriging var: {1}'.format(var_MC, var_kriging))

        df_indices_mc = create_df_from_indices(first_indices_mc)
        df_indices_gp = create_df_from_gp_indices(first_indices_gp)

        fig, axes = plt.subplots(3, 1, sharex=True, sharey=True, figsize=(8, 10))

        ax = axes[0]
        sns.violinplot(x='Variables', y='Indice values', data=df_indices_gp, ax=ax, gridsize=500)
        ax.set_xlabel('')
        ax.plot(ishigami_true_indices, 'yo', markersize=11, label='True indices')
        ax.set_title('Sobol GP Estimate')
        ax.legend(loc=0)
        ax.set_ylim(ylim)

        ax = axes[1]
        sns.violinplot(x='Variables', y='Indice values', data=df_indices_gp, hue='Error', ax=ax, split=True, gridsize=500)
        ax.set_xlabel('')
        ax.plot(ishigami_true_indices, 'yo', markersize=11, label='True indices')
        ax.set_title('Sobol GP Estimate - Kriging and MC errors')
        ax.legend(loc=0)
        ax.set_ylim(ylim)

        ax = axes[2]
        sns.violinplot(x='Variables', y='Indice values', data=df_indices_mc, ax=ax, split=True, gridsize=500)
        ax.set_xlabel('')
        ax.plot(ishigami_true_indices, 'yo', markersize=11, label='True indices')
        ax.set_title('Monte Carlo Estimate')
        ax.legend(loc=0)
        ax.set_ylim(ylim)

        fig.tight_layout()
        fig.savefig('output/ishigami_sobolGP_MC_n_%d_Nz_%d_m_%d_B_%d.pdf' % (max_budget, n_realization_gp, n_sample_gp, n_boot))
        fig.savefig('output/ishigami_sobolGP_MC_n_%d_Nz_%d_m_%d_B_%d.png' % (max_budget, n_realization_gp, n_sample_gp, n_boot))
        
        print('Elapsed time: %.2f s\n' % (time.time() - t_start))

n_sample_gp: 50, max_budget: 50
MC var: [ 0.10881896  0.14703915  0.1888995 ]
Kriging var: [ 0.13586829  0.15885839  0.1637971 ]
Elapsed time: 4.57 s

n_sample_gp: 50, max_budget: 100
MC var: [ 0.12457831  0.24418504  0.15878024]
Kriging var: [ 0.12774792  0.19175412  0.14556675]
Elapsed time: 3.67 s

n_sample_gp: 50, max_budget: 200
MC var: [ 0.13554775  0.72299689  0.15062067]
Kriging var: [ 0.11042748  0.46784881  0.09486512]
Elapsed time: 4.01 s

n_sample_gp: 50, max_budget: 500
MC var: [ 0.097583    0.12878935  0.18477888]
Kriging var: [ 0.04600092  0.05738183  0.06249274]
Elapsed time: 4.25 s

n_sample_gp: 50, max_budget: 1000
MC var: [ 0.09099623  0.11237869  0.16972456]
Kriging var: [ 0.01696236  0.02304128  0.0275996 ]
Elapsed time: 5.99 s

n_sample_gp: 100, max_budget: 50
MC var: [ 0.09164705  0.11817106  0.1309407 ]
Kriging var: [ 0.12899807  0.14592522  0.15721675]
Elapsed time: 4.67 s

n_sample_gp: 100, max_budget: 100
MC var: [ 0.10610475  0.09524425  0.08663538]
Kriging 

/scratch/ShapleySAMO/miniconda/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Elapsed time: 46.16 s

n_sample_gp: 1000, max_budget: 100
MC var: [ 0.03150072  0.03261923  0.0338883 ]
Kriging var: [ 0.04971693  0.04857852  0.03626381]
Elapsed time: 47.89 s

